# Giới thiệu sơ lược
##### Có thể bạn chưa biết, Pocket Monsters viết tắt là Pokemon, một series game cũng như anime đình đám, series game Pokemon phát triển bởi GameFreak hiện vẫn đang làm mưa làm gió trên các hệ máy Nitendo và trên thị trường game toàn cầu. Nhân dịp gen9 (Scarlet & Violet) của game vừa ra mắt, chúng ta sẽ điểm lại các pokemon từ những thế hệ trước và thông số cũng như đặc tính của chúng thông qua Pokedex. Ở đây, chúng ta sẽ sử dụng "https://pokemondb.net/pokedex/all" là trang web chứa dữ liệu Pokedex, hay nói cách khác là danh sách của toàn bộ Pokemon ĐÃ xuất hiện xuyên suốt series game này. Let's go!

# Install và import
- bs4 (Beautiful Soup): Một thư viện Python dùng để phân tích các tài liệu HTML và XML. Nó tạo ra một cây phân tích cú pháp cho các trang được phân tích cú pháp có thể được sử dụng để trích xuất dữ liệu từ HTML, rất hữu ích cho web scrapping.
- requests: Cho phép gửi các yêu cầu HTTP bằng Python.

In [ ]:
%pip install bs4
%pip install requests
%pip install plotly
%pip install numpy --upgrade
%pip install sklearn
%pip install tensorflow

In [ ]:
!cd

In [ ]:
import bs4
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf

# plotly packages
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# sklearn packages
from sklearn.model_selection import train_test_split

# tensorflow packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD

import warnings
warnings.filterwarnings("ignore")

# Thu thập và xử lý dữ liệu
### 1. Lấy danh sách các id riêng biệt của từng Pokemon
- **Lưu ý:** 

      - Có thể các Pokemon có rất nhiều dạng (vùng miền, mega evolution, mega X/mega Y, nguyên thủy (primal form), tấn công/phòng thủ,...), hình dạng Pokemon thay đổi nhưng id thì vẫn giữ nguyên, vì vậy sẽ xuất hiện trường hợp một id lặp lại nhiều lần, chúng ta chỉ xét hình dạng cơ bản của chúng nên chỉ cần lấy 1 id cho mỗi Pokemon thôi. 

In [ ]:
url = 'https://pokemondb.net/pokedex/all'

source = requests.get(url).text
soup = bs4.BeautifulSoup(source,'html.parser')

# Thu thập id của toàn bộ pokemon
pokemon_id = soup.findAll('span', {'class': 'infocard-cell-data'})

id = []

soup.findAll('span', {'class': 'infocard-cell-data'})
for i in range(len(pokemon_id)):
    id.append(pokemon_id[i].text)
    id[i] = int(id[i])

id = list(set(id))      # Sắp xếp và xóa các id bị trùng

### 2. Danh sách các url của từng Pokemon
- Chúng ta có thể dùng id hoặc tên của Pokemon để làm subdirectory của url tương ứng với Pokemon đó. 

##### Ví dụ: Pokemon có tên là Bulbasaur, id là 001:

https://pokemondb.net/pokedex/bulbasaur

https://pokemondb.net/pokedex/001 hay https://pokemondb.net/pokedex/1

In [ ]:
# Lấy danh sách urls cho web scrapping
urls = []
n = len(id)

for i in range(n):
    urls.append('https://pokemondb.net/pokedex/' + str(id[i]))

### 3. Thu thập dữ liệu
#### 3.1. Thông tin
- Lưu ý: ID của Pokemon sẽ được thêm vào sau cùng sau đó đặt ID làm cột index, ở đây chúng ta chỉ xét các trường như: tên, hệ, loài, chiều cao, cân nặng...
- Sẽ có thêm các cơ sở dữ liệu riêng để mô tả tác dụng của các đặc tính cũng như đặc tính ẩn và tương khắc hệ của chúng
- Các thông tin của một Pokemon sẽ bao gồm:

            - name: Tên

            - types_list: Hệ của Pokemon, mỗi con sẽ mang 1 hệ hoặc song hệ

            - species: Loài

            - height: Chiều cao

            - weight: Cân nặng

            - ability: Đặc tính

            - hid_ability (hidden ability): Đặc tính ẩn

            - catch_rate: Tỉ lệ bắt trúng bằng Poke Ball, bị ảnh hưởng bởi rất nhiều yếu tố:
                    * HP hiện tại
                    * Loại bóng (Poke Ball, Great Ball', Ultra Ball,...), Master Ball có tỷ lệ thu phục là 100% với tất cả Pokemon
                    * Trạng thái hiện tại (Choáng, phỏng, nhiễm độc, ngủ, tê liệt,...)
        
            - base_fs (base friendship): Chỉ số thân thiện cơ bản, một số Pokemon có chỉ số thân thiện là 0, bọn này có vẻ không ưa con người cho lắm

            - base_exp: Số lượng exp cần thiết để lên 1 level bất kỳ được tính theo hàm mũ. Mỗi loại có công thức khác nhau và khá phức tạp:
                    * Ví dụ 1 công thức đơn giản nhất cho mọi người:
                        Loại 1 000 000: exp = n^3 
                        n là level cần lên.
                        -> Để lên lv3 Pokemon loại này cần tổng cộng là 27exp trừ số exp hiện có sẽ ra số exp cần phải luyện thêm
            
            - growth_rate: Tốc độ tăng trưởng:
                    * Erratic: 600.000 exp ở level 100
                    * Fast: 800.000 điểm kinh nghiệm ở cấp 100
                    * Medium Fast: 1.000.000 điểm kinh nghiệm ở cấp 100
                    * Medium Slow: 1.059.860 điểm kinh nghiệm ở cấp 100
                    * Slow: 1.250.000 điểm kinh nghiệm ở cấp 100
                    * Fluctuating: 1.640.000 exp ở cấp 100:
        
            - genders: Giới tính theo tỷ lệ

            - Base Stats: Chỉ số cơ bản, bao gồm: 
                    * hp (Health Point): Sinh mệnh
                    * atk (Attack): Tấn công
                    * defn (Defense): Phòng thủ, giáp (Vì def là khai báo hàm nên không dùng để khai báo biến cho Defense được :v)
                    * sp_atk (Special Attack): Tấn công đặc biệt, sát thương phép
                    * sp_def (Special Defense): Phòng thủ đặc biệt, kháng phép
                    * spd (Speed): Tốc độ (tương tự như các game xoay tua, Pokemon tốc độ cao hơn sẽ được đánh trước)
                    
            - total: Tổng base stats


- Hàm get_pokemon_info(soup) giúp lấy ra thông tin của một Pokemon có ID chỉ định. Sau đó sẽ dùng vòng lặp để lấy thông tin toàn bộ Pokemon trong Pokedex

In [ ]:
# Thu thập dữ liệu nào
def get_pokemon_info(soup):
    # name
    name = soup.find('h1').text

    # types
    types_list = soup.find_all('td')[1].text.replace('\n', '')      # 'type1 type2 '
    types_list = types_list[:-1].split(' ')                         # 'type1 type2'

    # type1, type2
    type1 = types_list[0]
    
    if (len(types_list) == 2):   # Pokemon mang song hệ
        type2 = types_list[1]
    else: 
        type2 = ''              # Pokemon mang 1 hệ

    # species
    species = soup.find_all('td')[2].text

    # height
    height = soup.find_all('td')[3].text.replace('\xa0', '')

    # weight
    weight = soup.find_all('td')[4].text.replace('\xa0', '')

    # ability
    ability = soup.find_all('span', {'class': 'text-muted'})[0].text.replace('1. ', '')

    # extra ability
    if '2. ' in soup.findAll(class_='text-muted')[1].text:
        ex_ability = soup.findAll(class_='text-muted')[1].text.replace('2. ', '')
    else: 
        ex_ability = ''
        
    
    # hidden ability
    hid_ability = soup.find_all('small', {'class': 'text-muted'})[0].text.replace(' (hidden ability)', '')
    if '(' in hid_ability:  # Pokemon không có hidden ability nên bị đọc nhầm sang khu vực xuất hiện
        hid_ability = ''

    # catch_rate
    catch_rate = soup.find_all('td')[8].text.replace('\n', '')

    # base friendship 
    base_fs = soup.find_all('td')[9].text.replace('\n', '')

    # base exp
    base_exp = soup.find_all('td')[10].text.replace('\n', '')

    # growth_rate
    growth_rate = soup.find_all('td')[11].text.replace('\n', '')

    # genders
    genders = soup.find_all('td')[13].text.replace('\n', '')

    # base stats (base, min, max): health point, attack, defense, special attack, special defense, speed, total (base)
    hp = int(soup.find_all('td')[15].text.replace('\n', ''))

    atk = int(soup.find_all('td')[19].text.replace('\n', ''))

    defn = int(soup.find_all('td')[23].text.replace('\n', ''))

    sp_atk = int(soup.find_all('td')[27].text.replace('\n', ''))
    
    sp_def = int(soup.find_all('td')[31].text.replace('\n', ''))

    spd =  int(soup.find_all('td')[35].text.replace('\n', ''))

    total = int(soup.find_all('td')[39].text)

    return name, type1, type2, species, height, weight, ability, ex_ability, hid_ability, catch_rate, base_fs, base_exp, growth_rate, genders, hp, atk, defn, sp_atk, sp_def, spd, total

#### 3.2. Tương khắc hệ
- Tương khắc hệ (Type Defenses) của một Pokemon là khả năng chịu đòn của một Pokemon trước những đòn tấn công mang hệ khác. Các Pokemon Trainer sẽ dựa vào tương khắc hệ này của chúng nhằm chọn ra Pokemon có ưu thế về hệ (Type Advantages) tốt nhất để đối đầu với Pokemon của đối thủ.

        Ví dụ: Bulbasaur mang song hệ là Grass/Poison:
                - Fire -> Grass/Poison = 2 <=> Đòn đánh hệ Fire sẽ gây gấp đôi sát thương lên Pokemon này
                - Grass -> Grass/Poison = 1/4   <=> Đòn đánh hệ Grass sẽ gây 1/4 sát thương lên Pokemon này

- Hàm type_defense(soup) sẽ trả về tương khắc hệ của Pokemon có url tương ứng. 

In [ ]:
def type_defense(soup):
    type_defenses = []

    for i in range(40, 58):
        type_defenses.append(soup.findAll('td')[i].text)

    return type_defenses

#### 3.3. Đặc tính
- Đặc tính (Ability) và đặc tính ẩn (Hidden Ability): Là những đặc tính được trao cho mỗi Pokémon để có thể hỗ trợ chúng trong trận chiến, chúng ta sẽ lấy thông tin mô tả ngắn gọn toàn bộ đặc tính ở trang "https://pokemondb.net/ability".
- Một vài lợi ích đi kèm như: Thay đổi thời tiết, cường hóa tuyệt chiêu hoặc chỉ số, miễn nhiễm sát thương, giảm tấn công,...
 
        - Ví dụ: 
             * 'Intimidate' giảm chỉ số Attack của đối phương
             * 'Levitate' là trạng thái lơ lửng của Pokemon cho phép chúng miễn nhiễm với các đòn tấn công hệ Đất (Ground)
                
- Theo "https://pokemondb.net/ability", đặc tính và đặc tính ẩn đều gộp chung lại thành một danh sách được gọi là "The ability list".
- Danh sách này chứa:

                - Name: Tên đặc tính và đặc tính ẩn
                - Pokémon: Số lượng Pokemon sở hữu đặc tính đó
                - Description: Mô tả ngắn gọn về dặc tính đó
                - Gen: Thế hệ mà đặc tính đó được giới thiệu lần đầu

- Chúng ta chỉ cần quan tâm Name và Description.
- Hàm get_abilities_description(soup) trả về kết quả mô tả ngắn gọn về các Ability và Hidden Ability tương tứng.
- Lưu ý: 

        - Một vài Pokemon KHÔNG sở hữu Hidden Ability vì thế nội dung mô tả sẽ để trống
        - Hàm này chỉ trả về tên và mô tả các đặc tính ĐÃ ĐƯỢC GIỚI THIỆU XUYÊN SUỐT SERIES GAME POKEMON chứ không phải của từng Pokemon.

In [ ]:
def get_abilities_description():
    source = requests.get('https://pokemondb.net/ability').text
    soup = bs4.BeautifulSoup(source,'html.parser')
    
    n1 = len(soup.findAll('tr')) - 1
    ability = []
    ability_des = []

    for i in range(n1):
        ability.append(soup.find_all('a', {'class': 'ent-name'})[i].text)
        ability_des.append(soup.find_all('td', {'class': 'cell-med-text'})[i].text)

    # blanked values
    ability.append('')
    ability_des.append('')
    return {k: v for k, v in zip(ability, ability_des)}

#### 3.4. Bắt đầu lấy thông tin, tương khắc hệ, đặc tính của từng Pokemon

In [ ]:
data = []      # Thông tin
type_def = []  # Tương khắc hệ

for url in urls:
    source = requests.get(url).text
    soup = bs4.BeautifulSoup(source,'html.parser')
    # Lấy thông tin của từng Pokemon
    data.append(get_pokemon_info(soup))

    # Tương khắc hệ của từng Pokemon
    type_def.append(type_defense(soup))

# Mô tả đặc tính
abilities = get_abilities_description()

#### 3.4.1. Cơ sở dữ liệu thông tin Pokemon
- Sau khi đã có danh sách thông tin của toàn bộ Pokemon trong Pokedex, khởi tạo một cơ sở dữ liệu để lưu trữ
- Index ở đây sẽ là ID của từng Pokemon, bắt đầu từ 1.

In [ ]:
pokedex = pd.DataFrame(data = data, columns=['Name', 'Type 1', 'Type 2', 'Species', 'Height', 'Weight', 'Ability', 'Extra Ability', 'Hidden Ability',
                                             'Catch Rate', 'Base Friendship', 'Base Exp', 'Growth Rate', 'Gender',
                                             'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total'], index = id)
pokedex

#### 3.4.2. Cơ sở dữ liệu tương khắc hệ
- Tạo cơ sở dữ liệu để lưu trữ chỉ số tương khắc hệ.
- Chuẩn hóa dữ liệu dòng và thêm cột 'Name' chứa tên các Pokemon.

In [ ]:
# Tạo csdl chứa tương khắc hệ
type_defenses = pd.DataFrame(data = type_def, columns = ['NORMAL', 'FIRE', 'WATER', 'ELECTRIC', 'GRASS', 'ICE', 
                                        'FIGHTING', 'POISON', 'GROUND', 'FLYING', 'SPYCHIC', 'BUG',
                                        'ROCK', 'GHOST', 'DRAGON', 'DARK', 'STEEL', 'FAIRY'], index = id)

# Thay đổi giá trị các dòng cho dễ nhìn
wrong = ['', '½', '¼', '1½', '⅛']
right = ['1', '0.5', '0.25', '1.5', 1/3]

for i, j in zip(wrong, right):
    type_defenses = type_defenses.replace(i ,j)

# Thêm tên vào cơ dở dữ liệu
type_defenses.insert(0, 'Name', pokedex['Name'])
type_defenses.insert(1, 'Type 1', pokedex['Type 1'])
type_defenses.insert(2, 'Type 2', pokedex['Type 2'])

type_defenses

#### 3.4.3. Cơ sở dữ liệu mô tả các đặc tính
- Ý tưởng thực hiện:

                - B1: Trước tiên tạo database mới với tên, đặc tính, đặc tính ẩn Pokemon từ database cũ.
                - B2: Tạo 3 list là abi_des, ex_abi_des và hid_abi_des.
                - B3: Chạy vòng lặp từ 1 đến n+1 ứng với id từ 1 -> 1009  (n = 1008)
                - B4: Tạo các biến tạm để lưu giữ các Ability theo id
                - B5: abilities[temp] trả về mô tả ngắn gọn Ability tương ứng của Pokemon đó, tương tự như temp2 và temp3 với EA và HA.
                - B6: Tạo thêm 3 cột để chứa dữ liệu của 3 list.
                - B7: In csdl ra màn hình để kiểm tra.

In [ ]:
# Tạo csdl chứa tên và đặc tính tương ứng của từng Pokemon
abilities_info = pokedex[['Name', 'Ability', 'Extra Ability', 'Hidden Ability']]     # B1

# B2
abi_des = []        # ability description
ex_abi_des = []     # extra ability description
hid_abi_des = []    # hidden ability description

# B3
for i in range(1, n+1):
    # B4
    temp = abilities_info['Ability'][i]
    temp2 = abilities_info['Extra Ability'][i]
    temp3 = abilities_info['Hidden Ability'][i]
    
    # B5
    abi_des.append(abilities[temp])
    ex_abi_des.append(abilities[temp2])
    hid_abi_des.append(abilities[temp3])


# B6
abilities_info.insert(2, 'Ability Description', abi_des)
abilities_info.insert(4, 'Extra Ability Description', ex_abi_des)
abilities_info.insert(6, 'Hidden Ability Description', hid_abi_des)

# B7
abilities_info

# Xuất dữ liệu đã đọc được vào các file csv

In [ ]:
pokedex.to_csv('pokemon_db.csv', encoding='utf-8', index = True)
type_defenses.to_csv('type_defenses.csv', encoding='utf-8', index = True)
abilities_info.to_csv('abilities_description.csv', encoding='utf-8', index = True)

# Q & A

### 1. Các Pokemon phân bố theo hệ như thế nào? 

- Tất cả các Pokemon đều có 1 hoặc 2 hệ. Với các Pokemon có 1 hệ:

In [ ]:
data = pokedex['Type 1'].value_counts()
fig,ax = plt.subplots(figsize = (15,10))
ax.bar(data.keys(), data.values)

# xlabel, ylabel, suptitle
ax.set_xlabel('Hệ', fontsize=15)
ax.set_ylabel('Số lượng', fontsize=15)
plt.suptitle('\nSố lượng Pokemon theo hệ 1', fontsize=18)

# bar labels
for bars in ax.containers:
    ax.bar_label(bars, fontsize=14)

plt.show()

- Với các Pokemon mang song hệ, hệ 2 của chúng phân bố như sau:

In [ ]:
data1 = pokedex['Type 2'][pokedex['Type 2'] != ''].value_counts()

fig,ax = plt.subplots(figsize = (15,10))
ax.bar(data1.keys(), data1.values)

# xlabel, ylabel, suptitle
ax.set_xlabel('Hệ', fontsize=15)
ax.set_ylabel('Số lượng', fontsize=15)
plt.suptitle('\nSố lượng Pokemon theo hệ 2', fontsize=18)

# bar labels
for bars in ax.containers:
    ax.bar_label(bars, fontsize=14)

plt.show()

- Kết hợp cả 2 đồ thị lại, ta được:

In [ ]:
type1 = pokedex[['Type 1', 'Name']]\
                .rename(columns={'Type 1': 'Type', 'Name': 'Number'}) \
                .groupby('Type')\
                .count()

temp = pokedex[pokedex['Type 2'] != '']

type2 = temp[['Type 2', 'Name']]\
                .rename(columns={'Type 2': 'Type','Name': 'Number'}) \
                .groupby('Type')\
                .count()

xx = type1.index

fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.barh(xx, type1['Number'])
plt.barh(xx, type2['Number'], color = 'green')
plt.title('Thống kê Pokemon theo cả 2 hệ', fontsize = 15)
plt.legend('12')
plt.show()

In [ ]:
print('Số lượng Pokemon mang song hệ:', data1.sum(), '/' , data.sum())

- Nhận xét:
        
        - Số lượng Pokemon mang song hệ chiếm gần 50% tổng số Pokemon ĐÃ xuất hiện
        - Các hệ Water, Normal nhiều nhất ở hệ 1, nhưng lại ít xuất hiện hơn hẳn ở hệ 2.
        - Flying thường sẽ là hệ 2 của các Pokemon.
        - Một vài hệ có vẻ khá là hiếm ở Type 1, mặc dù đó là hệ phổ biến.

### 2. Pokemon mang hệ nào là tốt nhất? 

- Trừ các cột **Name, Type 1, Type 2** ra, chúng ta chỉ quan tâm chỉ số chống chịu của Pokemon với các hệ còn lại:

In [ ]:
best_types_list = pd.DataFrame()

cols = type_defenses.columns[3:]

for col in cols:
    best_types_list[col] = type_defenses[col].astype(float)

best_types_list

- Với `mean` càng cao thì sát thương gây ra cũng sẽ càng nhiều lên tất cả các hệ khác. Như vậy, top 5 hệ tốt nhất của chúng ta sẽ là:

In [ ]:
print(best_types_list.mean().sort_values(ascending=False)[0:5].round(2))
best_types_list.mean().sort_values(ascending=False)[0:5].plot(kind = 'bar', figsize = [8,5])

### 3. Cân nặng có liên quan gì đến khả năng chịu đòn của Pokemon hay không?

- Trước tiên Cân nặng của một Pokemon sẽ được tính theo `kg` và `lbs`, chúng ta sẽ quy về `kg` thông qua tiền xử lý:

In [ ]:
pokemon_weight = []

for i in range(1, n + 1):
    pokemon_weight.append(pokedex['Weight'].str.split('kg')[i][0])

pokemon_weight = list(map(float, pokemon_weight))

pokemon_weight

- Khả năng chịu đòn của một Pokemon được quyết định dựa trên 3 thuộc tính chủ yếu:

        + Điểm sinh mệnh (HP)
        + Phòng thủ hoặc giáp (Defense)
        + Phòng thủ đặc biệt hay kháng phép (Special Defense)

In [ ]:
# HP
plt.figure(figsize = (15,10))

sns.scatterplot(data = pokedex,
                x = pokedex['HP'], 
                y = pokemon_weight, 
                marker = 'o',
                s = 100). \
                set(xlabel = 'Pokemon HP', ylabel = 'Pokemon Weight', title = 'Weight - HP')

m, b = np.polyfit(pokedex['HP'], pokemon_weight, 1)
plt.plot(pokedex['HP'], m*pokedex['HP']+b)


# DEFENSE
plt.figure(figsize = (15,10))

sns.scatterplot(data = pokedex,
                x = pokedex['Defense'], 
                y = pokemon_weight, 
                marker = 'o',
                s = 100). \
                set(xlabel = 'Pokemon Defense', ylabel = 'Pokemon Weight', title = 'Weight - Defense')

m, b = np.polyfit(pokedex['Defense'], pokemon_weight, 1)
plt.plot(pokedex['Defense'], m*pokedex['Defense']+b)


# SPECIAL DEFENSE
plt.figure(figsize = (15,10))

sns.scatterplot(data = pokedex,
                x = pokedex['Sp. Def'], 
                y = pokemon_weight, 
                marker = 'o',
                s = 100). \
                set(xlabel = 'Pokemon Special Defense', ylabel = 'Pokemon Weight', title = 'Weight - Special Defense')

m, b = np.polyfit(pokedex['Sp. Def'], pokemon_weight, 1)
plt.plot(pokedex['Sp. Def'], m*pokedex['Sp. Def']+b)

- Nhận xét: 

        + Xu hướng tăng dần chỉ số phòng ngự theo cân nặng của Pokemon là có cơ sở
        + Các Pokemon "nặng ký" thì sẽ có khả năng chịu đòn càng tốt
        + Như vậy để có một Pokemon chịu đòn tốt, hãy chọn mấy con Pokemon mập :v

### 4. Nếu dựa trên tương khắc hệ có thể suy ra được hệ tốt nhất cho một Pokemon. Vậy dựa trên chỉ số sức mạnh thì các hệ nào sẽ đứng top? 

In [ ]:
strongest_types_list = pokedex[['Name', 'Type 1', 'Total']]. \
                        groupby('Type 1'). \
                        mean('Total'). \
                        sort_values('Total', ascending = False)

print(round(strongest_types_list, 2))

strongest_types_list[0:5].plot(kind = 'bar', figsize = [10,10], title = 'Top 5 hệ mạnh nhất', xlabel = 'Hệ', ylabel = 'Tổng chỉ số')

- Nhận xét:

        + DRAGON là hệ mạnh nhất, không có gì để bàn cãi cả.
        + FIRE và ROCK vừa là nằm trong top cả hệ tốt nhất và hệ mạnh nhất.

- Lưu ý:` Tuy DRAGON là hệ mạnh nhất nhưng vẫn không nằm trong các hệ tốt nhất là vì bọn này có chỉ số rất khủng, tuy vậy hệ tốt nhất chúng ta xét là dựa trên tương khắc hệ chứ không phải tổng chỉ số.`

### 5. Pokemon Á Thần, Huyền Bí và Huyền Thoại gồm những Pokemon nào?

- Trước hết, các Pokemon **Huyền Thoại (legendary)** ĐÃ được giới thiệu bao gồm những cái tên sau đây:

In [ ]:
# Mỗi dòng là các Pokemon huyền thoại tương ứng từ gen 1-9
legendaries = ['Articuno', 'Zapdos', 'Moltres', 'Mewtwo',
'Raikou', 'Entei', 'Suicune', 'Lugia', 'Ho-oh', 
'Regice', 'Registeel', 'Regirock', 'Latias', 'Latios', 'Groudon', 'Kyogre', 'Rayquaza', 
'Azelf', 'Mesprit', 'Uxie', 'Dialga', 'Palkia', 'Giratina', 'Cresselia', 'Heatran', 'Regigigas', 
'Tornadus', 'Thundurus', 'Landorus', 'Reshiram', 'Zekrom', 'Kyurem', 
'Cobalion', 'Terrakion', 'Virizion', 'Xerneas', 'Yveltal', 'Zygarde', 
'Type: Null', 'Silvally', 'Tapu Koko', 'Tapu Lele', 'Tapu Bulu', 'Tapu Fini', 'Cosmog', 'Cosmoem', 'Solgaleo', 'Lunala', 'Necrozma', 
'Zacian', 'Zamazenta', 'Eternatus', 'Kubfu', 'Urshifu', 'Regieleki', 'Regidrago', 'Glastrier', 'Spectrier', 'Calyrex',
'Koraidon', 'Miraidon', 'Chien-Pao', 'Ting-Lu', 'Wo-Chien', 'Chi-Yu']

pokedex['Legendary'] = pokedex['Name'].isin(legendaries)

pokedex[pokedex['Legendary'] == True]

- Các Pokemon **Huyền Bí (mythical)** ĐÃ được giới thiệu bao gồm những cái tên sau đây:

In [ ]:
# Mỗi dòng là các Pokemon huyền bí từ gen 1-9. Hiện tại gen 9 vẫn chưa thấy có Pokemon huyền bí nào
mythicals = ['Mew', 
'Celebi', 
'Jirachi', 'Deoxys', 
'Phione', 'Manaphy', 'Darkrai', 'Shaymin', 'Arceus', 
'Victini', 'Keldeo', 'Meloetta', 'Genesect', 
'Diancie', 'Hoopa', 'Volcanion', 
'Magearna', 'Marshadow', 'Zeraora', 'Meltan', 'Melmetal', 
'Zarude']

pokedex['Mythical'] = pokedex['Name'].isin(mythicals)

pokedex[pokedex['Mythical'] == True]

- Các Pokemon **Á Thần (pseudo-legendary)** được giới thiệu trong tiếng Anh là `The 600 club (câu lạc bộ 600)` ám chỉ các Pokemon KHÔNG phải là Pokemon Huyền Thoại, cũng chẳng phải Huyền Bí và có tổng chỉ số trên 600, trừ con `Slaking`:

In [ ]:
pokedex[pokedex['Legendary'] == False][pokedex['Mythical'] == False][pokedex['Total'] >= 600][pokedex['Name'] != 'Slaking']

In [ ]:
# Kiểm tra số lượng Pokemon huyền thoại, huyền bí trong Pokedex hiện tại có khớp với chiều dài danh sách không
print(pokedex[pokedex['Legendary'] == True].shape[0] == len(legendaries))
print(pokedex[pokedex['Mythical'] == True].shape[0] == len(mythicals))

### 6. So sánh chỉ số tổng quát của các Pokemon starter qua từng thời kỳ (gen 1 -> gen 9)?

In [ ]:
# Các dòng là starter Pokemon qua các thế hệ
starters =   ['Bulbasaur', 'Charmander', 'Squirtle', 
                'Chikorita', 'Cyndaquil', 'Totodile', 
                'Treecko', 'Torchic', 'Mudkip', 
                'Turtwig', 'Chimchar', 'Piplup',
                'Snivy', 'Tepig', 'Oshawott', 
                'Chespin', 'Fennekin', 'Froakie', 
                'Rowlet', 'Litten', 'Popplio',
                'Grookey', 'Scorbunny', 'Sobble',
                'Sprigatito', 'Fuecoco', 'Quaxly']



row = 9
col = 3   
idx = 0
fig = make_subplots(rows = row, cols = col, specs = [[{"type": "polar"}]*col]*row,
                   subplot_titles = tuple([pokemon for pokemon in starters]))

color = ['green', 'red', 'blue']

for i in range(row):
    for j in range(col):
        x = pokedex.query("Name == '{}'".format(starters[idx])).iloc[:,14:20]
        fig.add_trace(go.Scatterpolar(
        r = x.values[0].tolist(),
        theta = x.columns.tolist(),
        fill = 'toself',
        name = starters[idx], 
        fillcolor = color[j],
        opacity = 0.5),  row = i + 1, col = j + 1)
        idx += 1
    
fig.update_layout(height = 3000, showlegend = False, template = "plotly_dark")
fig.show()

### 7. Các cặp song hệ nào xuất hiện nhiều nhất?

- Chú ý:` Hệ 2 (Type 2)` và `Song hệ (Dual Types)` là 2 khái niệm khác nhau khác nhau:

    - `Type 2` là hệ thứ 2 của các Pokemon.

    - `Dual Types` là kết hợp `Type1` và `Type2`, `Type2` không phải là chuỗi rỗng.

- Trích xuất ra 10 song hệ phổ biến nhất:

In [ ]:
data = []

for i in range(1, n + 1):
    if pokedex['Type 2'][i] != '':
        data.append(pokedex['Type 1'][i] + ' & ' + pokedex['Type 2'][i])
    
Dual_Types = pd.Series(data = data).value_counts()[:10]

Dual_Types

- Trực quan hóa dữ liệu vừa trích xuất:

In [ ]:
ax = sns.barplot(y=Dual_Types.index, x=Dual_Types.values, orient='h', color='darkred')

ax.set_xlabel(xlabel='')
ax.set_xticklabels([])
ax.figure.set_size_inches(10, 8)

ax.set_yticklabels(ax.get_yticklabels(), size=14)

ax.set_title('Các cặp song hệ xuất hiện nhiều nhất', loc='center', pad=20, fontsize = 18)

sns.despine(top=True, right=True, left=True, bottom=True)

for index, value in enumerate(Dual_Types): 
    plt.annotate(f'{value}', xy=(value + 0.1, index), color='darkred') 

### 8. Các Pokemon mạnh nhất phân bố theo hệ như thế nào?

In [ ]:
type1 = pokedex['Type 1'].unique()

name = []

for type in type1:
    max = pokedex['Total'][pokedex['Type 1'] == type].max()
    name.append(pokedex['Name'][pokedex['Type 1'] == type][pokedex['Total'] == max].values)

pokedex[pokedex['Name'].isin(np.concatenate(name))].sort_values(by = 'Type 1')

# Mô hình hóa

## 1. Tiền xử lý (chuyển đổi kiểu dữ liệu)
#### a. Chuyển cân nặng sang kiểu float, đơn vị kilogram (kg)

In [ ]:
pokedex['Weight'] = pokemon_weight
pokedex.rename(columns = {'Weight': 'Weight (kg)'}, inplace = True)

#### b. Chuyển chiều cao sang kiểu float, đơn vị meter (m)

In [ ]:
pokemon_height = []

for i in range(1, n + 1):
    pokemon_height.append(pokedex['Height'].str.split('m')[i][0])

pokemon_height = list(map(float, pokemon_height))

In [ ]:
pokedex['Height'] = pokemon_height
pokedex.rename(columns = {'Height': 'Height (m)'}, inplace = True)

#### c. Base Exp, Base Friendship, Catch Rate

In [ ]:
# Base Exp
pokedex['Base Exp'] = list(map(int, pokedex['Base Exp']))

# Base Friendship
pokemon_fs = []

for i in range(1, n + 1):
    pokemon_fs.append(pokedex['Base Friendship'].str.split('(')[i][0])


# Catch Rate
pokemon_cr = []

for i in range(1, n + 1):
    pokemon_cr.append(pokedex['Catch Rate'].str.split('(')[i][0])


pokemon_fs = list(map(int, pokemon_fs))
pokemon_cr = list(map(int, pokemon_cr))
pokedex['Base Friendship'] = pokemon_fs
pokedex['Catch Rate'] = pokemon_cr

#### d. Chia pokemon theo thế hệ

In [ ]:
pokedex['Generation'] = 0
pokedex['Generation'][0:151] = 1
pokedex['Generation'][151:251] = 2
pokedex['Generation'][251:386] = 3
pokedex['Generation'][386:493] = 4
pokedex['Generation'][493:649] = 5
pokedex['Generation'][649:721] = 6
pokedex['Generation'][721:810] = 7
pokedex['Generation'][810:906] = 8
pokedex['Generation'][906:1008] = 9

## 2. Training dataset

#### a. Tiền xử lý
- Thiết lập các tập dữ liệu train/test

In [ ]:
features = ['Height (m)', 'Weight (kg)', 'Catch Rate', 'Base Friendship',
       'Base Exp', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',
       'Total', 'Legendary', 'Mythical', 'Generation']

id_features = pokedex.loc[:,~pokedex.columns.isin(features)].columns

# 0.9/0.1 train/test split
X_train, X_test = train_test_split(pokedex, test_size = 0.1)

input_shape = len(features)
print("Set shape:", X_train.shape, X_test.shape)
print("Input:",input_shape)

# 18 hệ
ntype = pokedex['Type 1'].nunique()
print("Type:", ntype)

y_train = X_train["Type 1"].values
y_test =  X_test["Type 1"].values

X_test_id = X_train.values[:, pokedex.columns.isin(id_features)]
X_train_id = X_test.values[:, pokedex.columns.isin(id_features)]

X_train = X_train.values[:, pokedex.columns.isin(features)]
X_test = X_test.values[:, pokedex.columns.isin(features)]

# Tiền xử lý X_train và X_test
scale = np.max(X_train)
X_train /= scale
X_test /= scale

print("Max: {}".format(scale))

- Mã hóa One-hot

In [ ]:
mapper_train, y_train_c = np.unique(y_train , return_inverse=True)
mapper_test, y_test_c = np.unique(y_test , return_inverse=True)

y_train_onehot = tf.keras.utils.to_categorical(y_train_c, num_classes = ntype)
y_test_onehot = tf.keras.utils.to_categorical(y_test_c, num_classes = ntype)

#### b. Mô hình hóa

- Mô hình `Sequential (Tuần tự)` thích hợp cho các lớp xếp chồng lên nhau, trong đó mỗi lớp chứa chính xác một input tensor và một output tensor.

In [ ]:
tf.keras.backend.clear_session()    # Reset model
model = Sequential()

# DNN
model.add(Dense(32, input_dim = input_shape, activation = 'relu'))
model.add(Dense(units = ntype, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy',
            optimizer = 'Adadelta',
            metrics = ['accuracy'])
            
model.summary()

In [ ]:
def gen_graph(history, title):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Accuracy ' + title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss ' + title)
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.legend(['train', 'validation'], loc='upper right')
    plt.show()

- Đào tạo 500 mẫu và xác thực:

In [ ]:
X_train = np.asarray(X_train).astype('float32')
history_rmsprop = model.fit(X_train, y_train_onehot, epochs = 500, validation_split = 0.1)

#### b. Trực quan hóa

In [ ]:
gen_graph(history_rmsprop, 
              "ResNet50 RMSprop")

In [ ]:
X_test = np.asarray(X_test).astype('float32')
y_pred  = model.predict(X_test)

y_classes = y_pred.argmax(axis=-1)
preds_df = pd.DataFrame({'#': X_train_id[:,0], 'Name': X_train_id[:,1], "Type_predicted": mapper_train[y_classes[:]]})

preds_df.to_csv('pokemons_prediction.csv')
preds_df.head()

scores = model.evaluate(X_test, y_test_onehot, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))